## **SETUP**

In [1]:
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
# SOME COMMON OPTIONS AND LIBRARIES THAT I USE  = = = = = = = = = = = = = 
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

import urllib.request
# pandas para manipulação de dataframes
import pandas as pd

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

# upload de arquivos 
from google.colab import files

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# Configurando o número máximo de linhas a se mostrar
pd.set_option('display.max_row', 5000)

# Configurando o número máximo de colunas a se mostrar
pd.set_option('display.max_columns', 50)

# Aumentando o número de caracteres a serem exibidos numa coluna de texto
pd.options.display.max_colwidth = 500

# Desligando a notificação setcopywarning
pd.set_option('mode.chained_assignment', None)

!pip install -U -q PyDrive
import os
import datetime as DT
import io
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import googleapiclient.discovery
import numpy as np 
import requests
import json
import time
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
import re

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# **RETRIEVE ALL LINKS FOR ATHLETICS IN SUMMER OLYMPICS PAGE**

In [2]:
###########################################################################################################
###
######### 1ª PARTE: PEGANDO TODOS OS LINKS DOS JOGOS OLÍMPICOS
###
###########################################################################################################

# RETRIEVE ALL LINKS FOR "ATHLETICS IN SUMMER OLYMPICS" ON WIKIPEDIA

res = requests.get("https://en.wikipedia.org/wiki/Athletics_at_the_Summer_Olympics")
soup = bs(res.text, "html.parser")
links = soup.select('a')
type(links)

athletic_links = []
for link in links:
    url = link.get("href", "")
    if "/wiki/Athletics" in url:

        d = {'url': url}         
        ooo = pd.DataFrame(d, index=[0])

        athletic_links.append(ooo) 

df_athletics = pd.concat(athletic_links).reset_index()
df_athletics

# NOW IT'S TIME TO FILTER AND RETAIN ONLY THOSE LINKS REGARDING OLYMPIC EDITIONS

df_athletics['link']     = df_athletics['url'].str.extract(r'(/wiki/Athletics_at_the_\d\d\d\d_Summer_Olympics_)*')
df_athletics2 = df_athletics.dropna(subset=['link']).reset_index()
df_athletics2 = df_athletics2[['url']]
df_athletics2.head(20)

# SOME REGEX CLEANNING

import re
df_athletics2['year']=df_athletics2['url'].str.extract(r'(\d{4})')
df_athletics2['year'] = df_athletics2['year'].astype(int)
df_athletics2.sort_values(['year'], ascending=[True], inplace=True)
df_athletics2 = df_athletics2.drop_duplicates()
#df_athletics2['type'] = df_athletics2['url'].str.extract(r'(?<=27s_).*')
df_athletics2['event'] = df_athletics2['url'].str.extract(r'(27s_[0-9A-za-z%,]+|Mixed_[0-9A-za-z%,]+).*')
df_athletics2['event'] = df_athletics2['event'].str.replace("27s_","")
df_athletics2['event'] = df_athletics2['event'].str.replace("%C3%97","x")
df_athletics2['event'] = df_athletics2['event'].str.replace("_"," ")
df_athletics2['men_women'] = df_athletics2['url'].str.extract(r'(Women|Men)') 

# LINKS FOR EVERY EVENT FROM 1896 TO 2021, MAN AND WOMAN
df_athletics2.head(5)

,url,year,event,men_women
0,/wiki/Athletics_at_the_1896_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1896,100 metres,Men
115,/wiki/Athletics_at_the_1896_Summer_Olympics_%E2%80%93_Men%27s_1500_metres,1896,1500 metres,Men
515,/wiki/Athletics_at_the_1896_Summer_Olympics_%E2%80%93_Men%27s_discus_throw,1896,discus throw,Men
86,/wiki/Athletics_at_the_1896_Summer_Olympics_%E2%80%93_Men%27s_800_metres,1896,800 metres,Men
486,/wiki/Athletics_at_the_1896_Summer_Olympics_%E2%80%93_Men%27s_shot_put,1896,shot put,Men


A LOOK AT THE 100M

In [3]:
df_athletics3 = df_athletics2.query('event == "100 metres"')
df_athletics3

,url,year,event,men_women
0,/wiki/Athletics_at_the_1896_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1896,100 metres,Men
1,/wiki/Athletics_at_the_1900_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1900,100 metres,Men
2,/wiki/Athletics_at_the_1904_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1904,100 metres,Men
3,/wiki/Athletics_at_the_1908_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1908,100 metres,Men
4,/wiki/Athletics_at_the_1912_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1912,100 metres,Men
5,/wiki/Athletics_at_the_1920_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1920,100 metres,Men
6,/wiki/Athletics_at_the_1924_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1924,100 metres,Men
695,/wiki/Athletics_at_the_1928_Summer_Olympics_%E2%80%93_Women%27s_100_metres,1928,100 metres,Women
7,/wiki/Athletics_at_the_1928_Summer_Olympics_%E2%80%93_Men%27s_100_metres,1928,100 metres,Men
696,/wiki/Athletics_at_the_1932_Summer_Olympics_%E2%80%93_Women%27s_100_metres,1932,100 metres,Women


# **BEAUTIFUL SOUP**

In [5]:
import time
#year <= 1904 &   & men_women == "Men"    & men_women == "Women" year == 1956 & 
#olympic_games = df_athletics2.query('year == 2020 & men_women=="Women" & event == "100 metres"')
olympic_games = df_athletics2.query('event == "100 metres"')
olympic_games

sleep_min = 1
sleep_max = 3
start_time = time.time()


table_bodies  = []
heats_column0 = []
all_tables0 = []
headers = []
table_titles = []
for olympics in olympic_games.itertuples():
    

    
    url0 = getattr(olympics, 'url') 
    url  = 'https://en.wikipedia.org/'+url0
    #print(url)
    
    year = getattr(olympics, 'year')
    print(year)
    men_women = getattr(olympics, 'men_women')    
    event = getattr(olympics, 'event')

    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")

    time.sleep(np.random.uniform(sleep_min, sleep_max))

    # ==================================================================================
    # 1 - PEGA TUDO QUE ESTÁ ENTRE O RESULTS E A TABELA FINAL. "É O QUE QUERO DA PÁGINA"
    # ==================================================================================

    heats = soup.find_all('span', {'class': 'mw-headline'})
    for i in heats:
      t = i.find_previous()
      h = t.find('span', {'class': 'mw-headline'}).text
      d = {'table_name': h, 'year': year, 'men_women': men_women}         
      ooo = pd.DataFrame(d, index=[0])
  
      table_titles.append(ooo)
      
    table_titles_final = pd.concat(table_titles).reset_index().drop(['index'], axis=1).reset_index()
    table_titles_final.rename(columns={"index": "id"}, inplace=True)
     
    table_titles_final['min_val'] = np.where( ((table_titles_final['table_name'] == "Results") | (table_titles_final['table_name'] == "Heats")), 1, np.nan)
    table_titles_final['max_val'] = np.where( ((table_titles_final['table_name'] == "Final")), 2,np.nan)
    table_titles_final['val'] = table_titles_final['min_val'].combine_first(table_titles_final['max_val']).fillna(0)#.astype(np.int64)
    table_titles_final = table_titles_final.drop(['min_val','max_val'], axis=1)
    table_titles_final

    # se conheço tabelas que atrapalham meu scrap e são desdobramentos das tabelas originais, posso apagá-las para não serem consideradas no meu código.
    #Por exemplo, Overall results nas olimpíadas de 2000; 100m F
    #também aproveito para limpar o nome das tabelas que contiverem ponto
    #table_titles_final2 = table_titles_final2[ table_titles_final2['table_name'].str.find("Overall") == -1 ]
    #table_titles_final2['table_name'] = table_titles_final2['table_name'].str.replace(".","")
    #table_titles_final2

    # ==================================================================================
    # 2 - PEGA TODOS OS HEADERS DE TODAS AS TABELAS QUE POSSUEM O MW-HEADLINE
    # ==================================================================================
             #class_ = "wikitable sortable" 
    table_data = soup.find_all('table', {"class":["wikitable sortable", "wikitable"]})
    #print(len(table_data))



    # para cada tabela do meu table data, quero o th, ou seja, cabeçalho
    for i in table_data:
        th = i.find_all('th')

        # e aí pego o texto do span anterior com a classe mw-headline
        table_name = i.find_previous('span', {'class': 'mw-headline'}).text
        #print(table_name)
        count = 0
        # Neste caso, como estou querendo empilhar cada cabeçalho de coluna em 
        # linhas separadas, faço um segundo loop para percorrer todas essas colunas, 
        # e aí monto o dataframe com o nome de cada tabela associada ao cabeçalho de 
        # coluna

        for j in th:
            if len(j.text.strip()) <= 1:              
              th2 = "apagar"
              #print("vazio", len(j), j)
            else:
              th2 = j.text.strip()
              count = count+1
              #print(len(j), j)
            d = {'table_name': table_name, 'column_order_id': count, 'column_name': th2, 'year': year, 'men_women': men_women}         
            ooo = pd.DataFrame(d, index=[0])
            headers.append(ooo)

headers_final = pd.concat(headers).reset_index().drop(['index'], axis=1).reset_index()

#assim como aqui na headers_final não queremos considerar esses headers
headers_final = headers_final[ headers_final['table_name'].str.find("Overall") == -1 ]

#também aproveito para limpar o nome das tabelas que contiverem ponto
headers_final['table_name'] = headers_final['table_name'].str.replace(".","")
headers_final['column_name'] = headers_final['column_name'].str.replace(".","")
headers_final['column_name'] = headers_final['column_name'].str.replace("Country","Nation")
headers_final
    # ==============================================================================

#year_genre = table_titles_final.query('men_women == "Women"')
table_titles_final['concat_for'] = table_titles_final['year'].astype(str) + table_titles_final['men_women']
table_titles_final

year_genre_loop = table_titles_final['concat_for'].unique().T
year_genre_loop = pd.DataFrame(year_genre_loop)
year_genre_loop.rename(columns={0: "year_genre"}, inplace=True)
year_genre_loop

right_tables = []

for ygenre in year_genre_loop.itertuples():
    
    filter = getattr(ygenre, 'year_genre')

    table_titles_final2 = table_titles_final.query('concat_for == @filter ')
    print(table_titles_final2)

    year = getattr(table_titles_final2, 'year')
    men_women = getattr(table_titles_final2, 'men_women')    
    val = getattr(table_titles_final2, 'val')    
    id = getattr(table_titles_final2, 'id')
    table_name = getattr(table_titles_final2, 'table_name')    

    if len(table_titles_final2.query('table_name=="Results"')) > 0:
      start = "Results"
    else:
      start = "Heats"

    df_min = table_titles_final2.query('table_name==@start')
    min_value = df_min['id'].values[0]
    min_value

    df_max = table_titles_final2.query('table_name=="Final"')
    max_value = df_max['id'].values[0]
    max_value

    table_titles_final3 = table_titles_final2.query('id >= @min_value & id<= @max_value ')
    right_tables.append(table_titles_final3)

tables_to_extract = pd.concat(right_tables)
tables_to_extract

table_scope0 = pd.merge(tables_to_extract, headers_final, left_on=['table_name', 'year', 'men_women'], right_on=['table_name', 'year', 'men_women'], how='inner').drop(['id', 'index'], axis=1)
table_scope = table_scope0[table_scope0.column_name != "apagar"]
table_scope.sort_index()
table_scope

table_scope = table_scope.drop(['val','concat_for'], axis=1)

#table_scope1 = table_scope[['table_name',	'year',	'men_women']]	
table_scope1 = table_scope[['year']].drop_duplicates()

table_append=[]

for row in table_scope1.itertuples():
    #print(row)
    year = getattr(row, 'year')

    filtered = table_scope1.query('year == @year')

    for row2 in filtered.itertuples():
      filtered_year = table_scope[['table_name','year','men_women']].query('year == @year')
      unique_combination = filtered_year.drop_duplicates()
      
      filtered_genre = unique_combination[['men_women']].drop_duplicates()

      for row3 in filtered_genre.itertuples():

        men_women = getattr(row3, 'men_women')

        genre = unique_combination.query('men_women == @men_women')

        #print(unique_combination)
        genre['id'] = 1
        genre['idx'] = genre.id.cumsum()
        table_append.append(genre)

append_final = pd.concat(table_append)
append_final

'''
table_scope.to_csv('/content/drive/My Drive/headers_final.csv')
table_titles_final2.to_csv('/content/drive/My Drive/table_titles.csv')
'''


1896
1900
1904
1908
1912
1920
1924
1928
1928
1932
1932
1936
1936
1948
1948
1952
1952
1956
1956
1960
1960
1964
1964
1968
1968
1972
1972
1976
1976
1980
1980
1984
1984
1988
1988
1992
1992
1996
1996
2000
2000
2004
2004
2008
2008
2012
2012
2016
2016
2020
2020


<ipython-input-5-fc7dab393d32>:104: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  headers_final['table_name'] = headers_final['table_name'].str.replace(".","")
<ipython-input-5-fc7dab393d32>:105: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  headers_final['column_name'] = headers_final['column_name'].str.replace(".","")


    id          table_name  year men_women  val concat_for
0    0          Background  1896       Men  0.0    1896Men
1    1  Competition format  1896       Men  0.0    1896Men
2    2             Records  1896       Men  0.0    1896Men
3    3            Schedule  1896       Men  0.0    1896Men
4    4             Results  1896       Men  1.0    1896Men
5    5               Heats  1896       Men  1.0    1896Men
6    6              Heat 1  1896       Men  0.0    1896Men
7    7              Heat 2  1896       Men  0.0    1896Men
8    8              Heat 3  1896       Men  0.0    1896Men
9    9               Final  1896       Men  2.0    1896Men
10  10     Results summary  1896       Men  0.0    1896Men
11  11          References  1896       Men  0.0    1896Men
    id          table_name  year men_women  val concat_for
12  12          Background  1900       Men  0.0    1900Men
13  13             Records  1900       Men  0.0    1900Men
14  14  Competition format  1900       Men  0.0    1900M

"\ntable_scope.to_csv('/content/drive/My Drive/headers_final.csv')\ntable_titles_final2.to_csv('/content/drive/My Drive/table_titles.csv')\n"

# **PANDAS HTML**

In [11]:
olympic_games = df_athletics2.query('event == "100 metres"')
olympic_games

sleep_min = 1
sleep_max = 3
start_time = time.time()

table_data  = []

for olympics in olympic_games.itertuples():
    
    url0 = getattr(olympics, 'url') 
    url  = 'https://en.wikipedia.org/'+url0
    #print(url)
    
    year = getattr(olympics, 'year')
    #print(year)
    
    men_women = getattr(olympics, 'men_women')    
    event = getattr(olympics, 'event')

    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")

    time.sleep(np.random.uniform(sleep_min, sleep_max))
                 
    table_MN = pd.read_html(url, match='Rank', attrs={'class': 'wikitable sortable'})
    #print(  len(table_MN), range(len(table_MN)), range(table_MN)  )
    for i in range(len(table_MN)):
        #print(i)
        #print(table_MN[i])

        for j in table_MN[i]:
          table_MN[i]['idx'] = i+1
          table_MN[i]['year'] = year
          table_MN[i]['men_women'] = men_women
          table_MN[i]['event'] = event

        table_data.append(table_MN[i])

df_extract = pd.concat(table_data)
df_extract       

print(len(df_extract))

base_final_trat0 = pd.merge(df_extract, append_final, left_on=['idx', 'year', 'men_women'], right_on=['idx', 'year', 'men_women'], how='inner', indicator=True)
base_final_trat0

# QUANDO FILTRAMOS A TABLE SCOPE PARA CONTER SOMENTE AS COLUNAS YEAR, MEN_WOMEN E TABLE_NAME, TIRANDO OS DUPLICADOS, CHEGAMOS NUMA TABELA QUE PODE SER 
# UTILIZADA COMO FILTRO PARA EVITAR QUE DETERMINADAS TABELAS GEREM LINHAS EM BRANCO. POR EXEMPLO, 1896 E 1904 A TABELA DE RESULTS SUMMARY APARECE COMO IDX5,
# MAS NÃO DEVERIA ESTAR ALI. COMO NA BASE_FINAL_TRAT 1 JÁ TEMOS O TABLE_NAME, SE FAZEMOS UM INNER JOIN "FILTRAMOS" SOMENTE AS ETAPAS (TABELAS) VÁLIDAS
# PARA CADA UMA DESSAS COMPETIÇÕES

#filtrando as colunas 
table_scope_filter = table_scope[['year','men_women','table_name']].drop_duplicates()
table_scope_filter

base_final_trat1 = base_final_trat0[['year'	,'men_women','table_name', 'Rank',	'Athlete'	,'Nation',	'Time',	'Notes'	,'idx'	,	'event']]

print(len(base_final_trat1))
print(len(table_scope_filter))
base_final_trat2 = pd.merge(base_final_trat1, table_scope_filter, left_on=['table_name', 'year', 'men_women'], right_on=['table_name', 'year', 'men_women'], how='inner', indicator=True)
base_final_trat2
print(len(base_final_trat2))

# FINAL TABLE:

base_final_trat2 = base_final_trat2.drop(['_merge'], axis=1)
base_right = pd.merge(base_final_trat2, df_extract, left_on=['idx', 'year', 'men_women'], right_on=['idx', 'year', 'men_women'], how='outer', indicator=True).query('_merge == "right_only"')
base_right.head(5)


5802
5624
782
5624


,year,men_women,table_name,Rank_x,Athlete_x,Nation_x,Time_x,Notes_x,idx,event_x,Rank_y,Athlete_y,Nation_y,Time_y,Notes_y,event_y,Semifinals,Final,Heats,Lane,Time (hand),Country,Time (automatic),Time (h),Time (a),Unnamed: 5,Heat,Reaction,Place,Qual.,Record,Unnamed: 7,Unnamed: 6,_merge
52482,1896,Men,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,Thomas Burke,United States,NaN,OR,100 metres,11.8,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
52483,1896,Men,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,Fritz Hofmann,Germany,NaN,NaN,100 metres,12.6,12.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
52484,1896,Men,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,Francis Lane,United States,NaN,NaN,100 metres,12.2,12.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
52485,1896,Men,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,Alajos Szokolyi,Hungary,NaN,NaN,100 metres,12.8,12.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
52486,1896,Men,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,5,Alexandros Chalkokondylis,Greece,NaN,NaN,100 metres,12.8,12.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
